# 函数

## 可接受任意数量参数的函数

In [1]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))

avg(1, 2, 3, 4)

2.5

In [2]:
import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
        name=name,
        attrs=attr_str,
        value=html.escape(value)
    )
    return element



In [3]:
make_element('item', 'Albatross', size='large', quantity=6)

'<item size="large" quantity="6">Albatross</item>'

In [4]:
make_element('p', '<spam>')

'<p>&lt;spam&gt;</p>'

## 只接受关键字参数的函数

In [6]:
def recv(maxsize, *, block):
    print('ok')

try:
    recv(1024,True)
except TypeError:
    print('Type Error')

recv(1024, block=True)

Type Error
ok


## 给函数参数添加元信息

In [1]:
def add(x:int, y:int) ->int:
    return x + y

help(add)

Help on function add in module __main__:

add(x: int, y: int) -> int



In [2]:
add.__annotations__

{'x': int, 'y': int, 'return': int}

## 返回多个值的函数

In [4]:
def myfun():
    return 1, 2, 3

a, b, c = myfun() # 返回的是一个元组
print(a, b, c)

1 2 3


## 定义有默认参数的函数

In [6]:
def spam(a, b=42):
    print(a, b)

spam(1)
spam(1, 2)

1 42
1 2


## 定义匿名或内联函数

In [7]:
add = lambda x, y : x + y
add(2, 3)

5

In [8]:
names = ['David Beazley', 'Brian Jones',
'Raymond Hettinger', 'Ned Batchelder']
sorted(names, key=lambda name : name.split()[-1].lower())

['Ned Batchelder', 'David Beazley', 'Raymond Hettinger', 'Brian Jones']

## 匿名函数捕获变量值

In [11]:
x = 10
a = lambda y: x + y
x = 20
b = lambda y: x + y

print(a(10))
print(b(10))
#  lambda 表达式中的 x 是一个自由变量，在运行时绑定值，而不是定义时就绑定

30
30


## 减少可调用对象的参数个数

In [12]:
def spam(a, b, c, d):
    print(a, b, c, d)

from functools import partial

s1 = partial(spam, 1)
s1(2, 3, 4)

s2 = partial(spam, d=42)
s2(1, 2, 3)

s3 = partial(spam, 1, 2, d=42)
s3(4)

1 2 3 4
1 2 3 42
1 2 4 42


In [13]:
points = [ (1, 2), (3, 4), (5, 6), (7, 8) ]
import math
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)

pt = (4, 3)
points.sort(key=partial(distance, pt))
points

[(3, 4), (1, 2), (5, 6), (7, 8)]

## 将单方法的类转换为函数

In [14]:
from urllib.request import urlopen
class UrlTemplate:
    def __init__(self, template):
        self.template = template
    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))

In [15]:
def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

## 带额外状态信息的回调函数

In [20]:
def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)

def print_result(result):
    print('Got: ', result)

add = lambda x, y : x + y

apply_async(add, (2,3), callback=print_result)

Got:  5


## 内联回调函数

In [33]:
from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


## 访问闭包中定义的变量

In [37]:
def sample():
    n = 0
    def func():
        print('n = ', n)
    
    def get_n():
        return n

    def set_n(value):
        nonlocal n
        n = value

    func.get_n = get_n
    func.set_n = set_n
    return func

f  = sample()
f()
f.set_n(10)
f()
f.get_n()

n =  0
n =  10


10